In [44]:
!pip install -q openai pypdf gradio

In [45]:
# Python modules:
import os
import re
from IPython.display import display, Markdown

# LLMs:
from openai import OpenAI

# Third-party & utils:
import pypdf
import gradio as gr

# Colab:
from google.colab import userdata

In [18]:
# Configuration of the API KEY and setup of the LLM client:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
client = OpenAI()

## Step 01: Input Loader

- User uploads resume (PDF, or Markdown).
- Enters job URL / Job Description text.
- Optional: interviewer name, LinkedIn, role.

In [29]:
def upload_resume_and_return_content(file):
  """
  This function reads the PDF fle uploaded and returns the .
  """
  pdf_text = ""

  pdf_reader = pypdf.PdfReader(file)
  pdf_pages = pdf_reader.pages

  for page in pdf_pages:
    pdf_text += page.extract_text()

  return pdf_text

def clean_pdf_text(text):
  """
  Remove multiple lines and spaces from the text.
  """
  text = re.sub(r'\n{2,}', '\n', text)
  text = re.sub(r'[ ]{2,}', ' ', text)
  return text.strip()

In [36]:
# User resume:
raw_user_resume = upload_resume_and_return_content("./Leandro_2025.pdf")
user_resume = clean_pdf_text(raw_user_resume)

In [61]:
# JSON description:
prompt = {
  "job_url": "",
  "job_description": "",
  "interviewer_name": "",
  "interviewer_linkedin": "",
  "interviewer_role": "",
  "interviewer_experience": ""
}

## Step 02: Interview Simulation Engine

- Generates tailored questions based on job and resume
- Handles real-time interaction (text initially, voice later)
- Supports behavioral & technical rounds



## Step 03: Feedback Engine

- Analyzes each answer using:
  - STAR structure recognition
  - Clarity/relevance rubric
- Highlights strengths, flags issues


## Step 04: Replay & Summary

- Transcript of Q&A
- Highlight key moments
- Score each question and tag areas for improvement


## Step 05: Progress Tracker

- Stores past mock sessions
- Tracks performance over time

In [66]:
# Gradio UI:
def build_prompt(job_url, job_description, interviewer_name, interviewer_linkedin, interviewer_role, interviewer_experience):
    return {
        "job_url": job_url,
        "job_description": job_description,
        "interviewer_name": interviewer_name,
        "interviewer_linkedin": interviewer_linkedin,
        "interviewer_role": interviewer_role,
        "interviewer_experience": interviewer_experience
    }


with gr.Blocks() as app_ui:
    gr.Markdown("# Job Description and Interviewer Information")
    gr.Markdown("----")

    upload_button = gr.UploadButton("Upload Resume", file_types=[".pdf"])
    user_resume = gr.Textbox(label="User Resume", interactive=False)
    upload_button.upload(upload_resume_and_return_content, upload_button, user_resume)

    gr.Markdown("----")
    job_url = gr.Textbox(label="Job URL")
    job_description = gr.Textbox(label="Job Description", lines=5)

    gr.Markdown("----")
    interviewer_name = gr.Textbox(label="Interviewer Name")
    interviewer_linkedin = gr.Textbox(label="Interviewer LinkedIn")
    interviewer_role = gr.Textbox(label="Interviewer Role")
    interviewer_experience = gr.Textbox(label="Interviewer Experience")

    gr.Markdown("----")
    submit_btn = gr.Button("Build Prompt JSON")
    prompt_output = gr.JSON(label="Prompt JSON Output")

    submit_btn.click(
        build_prompt,
        inputs=[
            job_url,
            job_description,
            interviewer_name,
            interviewer_linkedin,
            interviewer_role,
            interviewer_experience
        ],
        outputs=prompt_output
    )

app_ui.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f1f8c1748389dbfcc4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
